# Neural Collaborative Quick Filtering on MovieLens dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")
import os
import shutil
import papermill as pm
import pandas as pd
import numpy as np
import tensorflow as tf

from reco_utils.common.timer import Timer
from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import (rmse, ndcg_at_k, precision_at_k, recall_at_k)
from sklearn.model_selection import train_test_split

#We are using the same environment with the pacakage provider-Microsoft
print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

/Users/zoe/anaconda3/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zoe/anaconda3/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zoe/anaconda3/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zoe/anaconda3/envs/reco_base/lib/pyt

System version: 3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 18:53:43) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas version: 0.25.3
Tensorflow version: 1.12.0


In [3]:
# top k items to recommend
TOP_K = 5

# Model parameters
EPOCHS = 30
BATCH_SIZE = 256

SEED = 4850

In [4]:
lastfm = pd.read_csv("user_artists.csv")
lastfm.rename(columns={"userID": "userID", "artistID": "itemID","weight" :"timestamp"}, 
                 inplace=True)
lastfm = lastfm.drop(lastfm.columns[[0]], axis=1)

In [5]:
print(lastfm.shape)

(92834, 4)


In [6]:
#Take a look at this dataset
lastfm.head()

,userID,itemID,timestamp,rating
0,2,51,13883,5.5
1,2,52,11690,4.6
2,2,53,11351,4.5
3,2,54,10300,4.1
4,2,55,8983,3.6


In [7]:

len(lastfm['userID'].unique()),len(lastfm['itemID'].unique()),len(lastfm['rating']),len(lastfm['timestamp'])

(1892, 17632, 92834, 92834)

In [8]:
train, test = python_chrono_split(lastfm, 0.75)

In [9]:
#Generate an NCF dataset object from the data subsets.
data = NCFDataset(train=train, test=test, seed=SEED)

### 3.3 Train NCF based on TensorFlow
The NCF has a lot of parameters. The most important ones are:

`n_factors`, which controls the dimension of the latent space. Usually, the quality of the training set predictions grows with as n_factors gets higher.

`layer_sizes`, sizes of input layer (and hidden layers) of MLP, input type is list.

`n_epochs`, which defines the number of iteration of the SGD procedure.
Note that both parameter also affect the training time.

`model_type`, we can train single `"mlp"`, `"gmf"` or combined model `"neumf"` by changing the type of model.

We will here set `n_factors` to `10`, `layer_sizes` to `[16,8,4]`,  `n_epochs` to `30`, `batch_size` to 256. To train the model, we simply need to call the `fit()` method.

In [10]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="neumf",
    n_factors=10,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [11]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

Took 418.0828064140005 seconds for training.


### 3.4.1 Prediction


In [12]:
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]


predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

,userID,itemID,prediction
0,2.0,62.0,2.757727e-04
1,2.0,61.0,9.992127e-01
2,2.0,60.0,2.461172e-09
3,2.0,59.0,9.990662e-01
4,2.0,58.0,9.989328e-01


### 3.4.2 Generic Evaluation

In [13]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 53.71547543899942 seconds for prediction.


In [14]:
eval_rmse = rmse(test, all_predictions, col_prediction='prediction')
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("RMSE:\t%f" % eval_rmse,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

RMSE:	2.155824
NDCG:	0.172741
Precision@K:	0.166968
Recall@K:	0.069990


## 3.5 Pre-training
We are going to use pre-training to get a better performance

##### Pre-training GMF Model

In [15]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="GMF",
    n_factors=10,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [16]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

model.save(dir_name=".pretrain/GMF")

Took 360.32037220799975 seconds for training.


In [17]:
#We also curios about how GMF performance by itself
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 50.90504190399952 seconds for prediction.


In [18]:
eval_rmse = rmse(test, all_predictions, col_prediction='prediction')
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("RMSE:\t%f" % eval_rmse,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

RMSE:	2.147799
NDCG:	0.168483
Precision@K:	0.164631
Recall@K:	0.069557


##### Pre-training MLP Model

###MLP

In [19]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="MLP",
    n_factors=10,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [20]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

model.save(dir_name=".pretrain/MLP")

Took 365.80841789199985 seconds for training.


In [21]:
#We also curios about how MLP performance by itself
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 51.01154921399939 seconds for prediction.


In [22]:
eval_rmse = rmse(test, all_predictions, col_prediction='prediction')
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("RMSE:\t%f" % eval_rmse,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

RMSE:	2.175382
NDCG:	0.138652
Precision@K:	0.136484
Recall@K:	0.057070


### 3.5.2 Load pre-trained GMF and MLP model for NeuMF

In [23]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=10,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)


#We give more weight for mlp
model.load(gmf_dir=".pretrain/GMF", mlp_dir=".pretrain/MLP", alpha=0.4)

INFO:tensorflow:Restoring parameters from .pretrain/GMF/model.ckpt
INFO:tensorflow:Restoring parameters from .pretrain/MLP/model.ckpt


In [24]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

Took 437.59053736400165 seconds for training.


In [25]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 57.80752462599776 seconds for prediction.


In [26]:
eval_rmse = rmse(test, all_predictions, col_prediction='prediction')
eval_ndcg2 = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision2 = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall2 = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("RMSE:\t%f" % eval_rmse,
      "NDCG:\t%f" % eval_ndcg2,
      "Precision@K:\t%f" % eval_precision2,
      "Recall@K:\t%f" % eval_recall2, sep='\n')

RMSE:	2.157456
NDCG:	0.129670
Precision@K:	0.129474
Recall@K:	0.054132
